In [ ]:
import pandas as pd
import numpy as np
from itertools import chain

# Hi-C utilities imports:
import cooler
import bioframe
import cooltools
from cooltools.lib.numutils import fill_diag
from cooltools.lib import numutils

# Visualization imports:
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.colors import PowerNorm
from matplotlib import cm
# %matplotlib widget
%matplotlib inline

import matplotlib.patches as patches
from matplotlib.ticker import EngFormatter
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.transforms import Affine2D
from cooltools.lib import plotting

from itertools import cycle

# # from ipywidgets import interact, fixed
# from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets

# enable editable text ...
mpl.rcParams["pdf.fonttype"]=42
mpl.rcParams["svg.fonttype"]="none"
mpl.rcParams['axes.linewidth'] = 0.5

### Import modified "guts" of the dotfinder submodule from `helper_func` file

In [ ]:
from helper_func import draw_kernel

# turns out still need some of the dotfinder guts in here
from cooltools.api.dotfinder import bp_to_bins, generate_tiles_diag_band
from cooltools.lib.numutils import LazyToeplitz
from cooltools.lib.common import assign_regions

from functools import partial
from data_catalog import bws, bws_vlim, telo_dict

import datashader as ds, xarray as xr
from datashader import transfer_functions as tf, reductions as rd
from datashader.mpl_ext import dsshow, alpha_colormap

from matplotlib.cm import YlOrRd
import matplotlib.image as mpimg

from palettable.scientific import sequential
lajolla_r = sequential.LaJolla_10_r.get_mpl_colormap()
bilbao_r = sequential.Bilbao_20_r.get_mpl_colormap()
batlow_r = sequential.Batlow_20_r.get_mpl_colormap()
devon_r = sequential.Devon_20_r.get_mpl_colormap()
davos_r = sequential.Davos_20_r.get_mpl_colormap()
# fall_cmap = fall
lajolla = sequential.LaJolla_10.get_mpl_colormap()
bilbao = sequential.Bilbao_20.get_mpl_colormap()
batlow = sequential.Batlow_20.get_mpl_colormap()
devon = sequential.Devon_20.get_mpl_colormap()
davos = sequential.Davos_20.get_mpl_colormap()

imola_r = sequential.Imola_20_r.get_mpl_colormap()
imola = sequential.Imola_20.get_mpl_colormap()

nuuk_r = sequential.Nuuk_20_r.get_mpl_colormap()
nuuk = sequential.Nuuk_20.get_mpl_colormap()

hawaii_r = sequential.Hawaii_20_r.get_mpl_colormap()
hawaii = sequential.Hawaii_20.get_mpl_colormap()

oleron_r = sequential.Oleron_20_r.get_mpl_colormap()
oleron = sequential.Oleron_20.get_mpl_colormap()

oslo_r = sequential.Oslo_20_r.get_mpl_colormap()
oslo = sequential.Oslo_20.get_mpl_colormap()

bamako_r = sequential.Bamako_20_r.get_mpl_colormap()
bamako = sequential.Bamako_20.get_mpl_colormap()

grayc_r = sequential.GrayC_20_r.get_mpl_colormap()
grayc = sequential.GrayC_20.get_mpl_colormap()

lapaz_r = sequential.LaPaz_20_r.get_mpl_colormap()
lapaz = sequential.LaPaz_20.get_mpl_colormap()


In [ ]:
import numpy as np
from scipy import interpolate
# https://stackoverflow.com/questions/6518811/interpolate-nan-values-in-a-numpy-array
def fill_nan(A):
    '''
    interpolate to fill nan values
    '''
    inds = np.arange(A.shape[0])
    good = np.where(np.isfinite(A))
    f = interpolate.interp1d(inds[good], A[good],bounds_error=False)
    B = np.where(np.isfinite(A),A,f(inds))
    return B

### pick a dataset and binsize to work on ...

In [ ]:
# ! ls /data/proj_sync/data_ranger/finalcoolers_mega/ranGAP1-aux-G1s-MEGA.hg38.mapq_30.1000.mcool
#
# 10 kb is a resolution at which one can clearly see "dots":
binsize10 = 10_000
# cooler files that we'll work on :
telo_clrs10 = { _k: cooler.Cooler(f"{_path}::/resolutions/{binsize10}") for _k, _path in telo_dict.items() }

# 10 kb is a resolution at which one can clearly see "dots":
binsize50 = 50_000
# cooler files that we'll work on :
telo_clrs50 = { _k: cooler.Cooler(f"{_path}::/resolutions/{binsize50}") for _k, _path in telo_dict.items() }

# 10 kb is a resolution at which one can clearly see "dots":
binsize250 = 250_000
# cooler files that we'll work on :
telo_clrs250 = { _k: cooler.Cooler(f"{_path}::/resolutions/{binsize250}") for _k, _path in telo_dict.items() }

# Working on a Figure here

250kb (incl trans)
 - region1 = ('chr6', 105000000, 155000000)
 - region2 = ('chr7', 0, 56000000)

50kb
 - ('chr6',125200000, 145200000)

10kb assembly on- and off-diagonal
 - on diagonal #1: ('chr6', 129100000, 130500000)
 - on diagonal #2: ('chr6', 136100000, 138200000)

In [ ]:
mycmap = mpl.colors.LinearSegmentedColormap.from_list(
    name="myfall",
    colors=[ mpl.colors.to_hex(c) for c in plotting.PALETTES["fall"] ],
    N=2**16,
)
mycmap.set_bad('white',.5)
mycmap

In [ ]:
_chr6_len = bioframe.fetch_chromsizes("hg38")["chr6"]
region1_250 = ('chr6', 105000000, _chr6_len)
region2_250 = ('chr7', 0, 56000000)
# ...
region1_50 = ('chr6', 125200000, 145200000)
# ...
region1_10 = ('chr6', 129100000, 130500000)
region2_10 = ('chr6', 136100000, 138200000)

In [ ]:
_chr6_len = bioframe.fetch_chromsizes("hg38")["chr6"]
# region1_250 = ('chr6', 105000000+25_000_000, _chr6_len)
# region2_250 = ('chr7', 0, 56000000-35_000_000)
# # ...
# region1_50 = ('chr6', 125200000+1_500_000, 145200000-4_500_000)
# # ...
region1_10 = ('chr6', 129100000-30_000, 130500000+30_000)
region2_10 = ('chr6', 136100000-45_000, 138200000+45_000)

### define arms for the schematic ...

In [ ]:
# Use bioframe to fetch the genomic features from the UCSC.
hg38_chromsizes = bioframe.fetch_chromsizes('hg38')
hg38_cens = bioframe.fetch_centromeres('hg38')
hg38_arms_full = bioframe.make_chromarms(hg38_chromsizes, hg38_cens)
arms_dict = hg38_arms_full.set_index("name").eval("(end - start)//1000000").to_dict()

In [ ]:

# def get_hm(
#     clr,
#     _idx_from,
#     _idx_to,
#     _jdx_from=None,
#     _jdx_to=None,
#     vmin=0.0002,
#     vmax=0.0125,
# ):
#     #
#     _jdx_from = _idx_from if _jdx_from is None else _jdx_from
#     _jdx_to = _idx_to if _jdx_to is None else _jdx_to
#     _mat = xr.DataArray(
#         clr.matrix()[_idx_from:_idx_to, _jdx_from:_jdx_to]
#     )
#     return tf.shade(
#         _mat,
#         cmap = mycmap,
#         # how = lambda x,y: np.power(x, 1/3.),
#         how = lambda x,y: np.log10(x),
#         span = (vmin, vmax),
#     ).to_pil(origin="upper")


# switch back to s
def get_hm(
    clr,
    _idx_from,
    _idx_to,
    _jdx_from=None,
    _jdx_to=None,
    vmin=0.0002,
    vmax=0.0125,
):
    #
    _jdx_from = _idx_from if _jdx_from is None else _jdx_from
    _jdx_to = _idx_to if _jdx_to is None else _jdx_to
    _mat = clr.matrix()[_idx_from:_idx_to, _jdx_from:_jdx_to]
    # need to NaN-out 2 diagonals ...
    if _idx_from == _jdx_from:
        for i in [-1,0,1]:
            _mat = numutils.fill_diag(_mat, np.nan, i)
        # ...
    _imshow_kwargs = dict(
        norm = LogNorm(vmin=vmin, vmax=vmax),
        cmap = mycmap,
        interpolation="nearest",
    )
    return _mat, _imshow_kwargs




# vlims_dict = {
#     "250": dict(vmin=0.000005, vmax=0.002),
#     "50": dict(vmin=0.0001, vmax=0.01),
#     "10_1": dict(vmin=0.0002, vmax=0.0125),
#     "10_2": dict(vmin=0.0001, vmax=0.0025),
# }

vlims_dict = {
    "250": dict(vmin=0.00001, vmax=0.003),
    "50": dict(vmin=0.00004, vmax=0.025),
    # "10_1": dict(vmin=0.00006, vmax=0.06),
    # "10_2": dict(vmin=0.00006, vmax=0.06),
    "10_1": dict(vmin=0.0001, vmax=0.01),
    "10_2": dict(vmin=0.0001, vmax=0.01),
}


In [ ]:
# import matplotlib.pyplot as plt
import bbi
import matplotlib.lines as lines
from matplotlib.lines import Line2D
from matplotlib.patches import ConnectionPatch, Rectangle
from mpl_toolkits.axes_grid1 import Divider, Size
from mpl_toolkits.axes_grid1.inset_locator import BboxConnector

# manual colorbar example ...
# #
# cax = divider.append_axes("right", size=0.1, pad=0.05)
# plt.colorbar(
#     cm.ScalarMappable(norm=PowerNorm(1/3., vmin=vmin, vmax=vmax), cmap=mycmap),
#     cax=cax,
# )

import matplotlib.patches as mpatches
import matplotlib.path as mpath

# import matplotlib.pyplot as plt
import bbi
import matplotlib.lines as lines
from matplotlib.lines import Line2D
from matplotlib.patches import ConnectionPatch, Rectangle
from mpl_toolkits.axes_grid1 import Divider, Size
from mpl_toolkits.axes_grid1.inset_locator import BboxConnector


# draw 2 arm chromosome ....
def get_chrom_path(
    x0y0,  # position of the centromere ...
    parm,  # p arm length
    qarm,  # q arm length
    width,  # width
    color="grey",
    **patch_kwargs,
):
    # unpack coordinates of the center (centromere)
    x0, y0 = x0y0

    # calculate half width
    hw = 0.5*width

    dtelo = 0.5*width
    dxcent = 0.2*width
    dycent = 0.5*width

    # Prepare the data for the PathPatch below.
    Path = mpath.Path
    codes, verts = zip(*[
        # p-arm up:
        (Path.MOVETO, [x0-hw, y0+dycent]),
        (Path.LINETO, [x0-hw, y0+parm-dtelo]),
        # p-telomere curve:
        (Path.CURVE3, [x0-hw, y0+parm]),
        (Path.CURVE3, [x0, y0+parm]),
        (Path.CURVE3, [x0+hw, y0+parm]),
        (Path.CURVE3, [x0+hw, y0+parm-dtelo]),
        # p-arm down:
        (Path.LINETO, [x0+hw, y0+dycent]),
        # centromere S curve (right)
        # (1S)
        (Path.CURVE4, [x0+hw, y0+0.5*dycent]),
        (Path.CURVE4, [x0+hw-dxcent, y0+0.5*dycent]),
        (Path.CURVE4, [x0+hw-dxcent, y0]),
        # (2S)
        (Path.CURVE4, [x0+hw-dxcent, y0-0.5*dycent]),
        (Path.CURVE4, [x0+hw, y0-0.5*dycent]),
        (Path.CURVE4, [x0+hw, y0-dycent]),
        # q-arm down
        (Path.LINETO, [x0+hw, y0-qarm+dtelo]),
        # q-telomere curve:
        (Path.CURVE3, [x0+hw, y0-qarm]),
        (Path.CURVE3, [x0, y0-qarm]),
        (Path.CURVE3, [x0-hw, y0-qarm]),
        (Path.CURVE3, [x0-hw, y0-qarm+dtelo]),
        # q-arm up
        (Path.LINETO, [x0-hw, y0-dycent]),
        # centromere S curve (left)
        (Path.CURVE4, [x0-hw, y0-0.5*dycent]),
        (Path.CURVE4, [x0-hw+dxcent, y0-0.5*dycent]),
        (Path.CURVE4, [x0-hw+dxcent, y0]),
        (Path.CURVE4, [x0-hw+dxcent, y0+0.5*dycent]),
        (Path.CURVE4, [x0-hw, y0+0.5*dycent]),
        (Path.CURVE4, [x0-hw, y0+dycent]),
        (Path.CLOSEPOLY, [x0-hw, y0+dycent]),
    ])

    chrom = mpatches.PathPatch(mpath.Path(verts, codes), ec="none", **patch_kwargs)
    chrom.set(color=color)
    return chrom


# draw 2 arm chromosome ....
def get_chrom_path_hor(
    x0y0,  # position of the centromere ...
    parm,  # p arm length
    qarm,  # q arm length
    width,  # width
    color="grey",
):
    # unpack coordinates of the center (centromere)
    x0, y0 = x0y0

    # calculate half width
    hw = 0.5*width

    dtelo = 0.5*width
    dxcent = 0.2*width
    dycent = 0.5*width

    # Prepare the data for the PathPatch below.
    Path = mpath.Path
    codes, verts = zip(*[
        # p-arm up:
        (Path.MOVETO, [x0-hw, y0+dycent] ),
        (Path.LINETO, [x0-hw, y0+parm-dtelo] ),
        # p-telomere curve:
        (Path.CURVE3, [x0-hw, y0+parm] ),
        (Path.CURVE3, [x0, y0+parm] ),
        (Path.CURVE3, [x0+hw, y0+parm] ),
        (Path.CURVE3, [x0+hw, y0+parm-dtelo] ),
        # p-arm down:
        (Path.LINETO, [x0+hw, y0+dycent] ),
        # centromere S curve (right)
        # (1S)
        (Path.CURVE4, [x0+hw, y0+0.5*dycent] ),
        (Path.CURVE4, [x0+hw-dxcent, y0+0.5*dycent] ),
        (Path.CURVE4, [x0+hw-dxcent, y0] ),
        # (2S)
        (Path.CURVE4, [x0+hw-dxcent, y0-0.5*dycent] ),
        (Path.CURVE4, [x0+hw, y0-0.5*dycent] ),
        (Path.CURVE4, [x0+hw, y0-dycent] ),
        # q-arm down
        (Path.LINETO, [x0+hw, y0-qarm+dtelo] ),
        # q-telomere curve:
        (Path.CURVE3, [x0+hw, y0-qarm] ),
        (Path.CURVE3, [x0, y0-qarm] ),
        (Path.CURVE3, [x0-hw, y0-qarm] ),
        (Path.CURVE3, [x0-hw, y0-qarm+dtelo] ),
        # q-arm up
        (Path.LINETO, [x0-hw, y0-dycent] ),
        # centromere S curve (left)
        (Path.CURVE4, [x0-hw, y0-0.5*dycent] ),
        (Path.CURVE4, [x0-hw+dxcent, y0-0.5*dycent] ),
        (Path.CURVE4, [x0-hw+dxcent, y0] ),
        (Path.CURVE4, [x0-hw+dxcent, y0+0.5*dycent] ),
        (Path.CURVE4, [x0-hw, y0+0.5*dycent] ),
        (Path.CURVE4, [x0-hw, y0+dycent] ),
        (Path.CLOSEPOLY, [x0-hw, y0+dycent] ),
    ])

    from matplotlib.transforms import Affine2D
    verts = Affine2D().rotate_deg_around(x0, y0, 90).transform(verts)
    chrom = mpatches.PathPatch(mpath.Path(verts, codes), ec="none")
    chrom.set(color=color)
    return chrom





# draw 2 arm chromosome ....
def get_chrom_path_wpathes(
    x0y0,  # position of the centromere ...
    parm,  # p arm length
    qarm,  # q arm length
    width,  # width
    color="grey",
    patches=None,  # [(_arm, _from, _to, pcolor)]
    rotation_deg = 0,
):
    # unpack coordinates of the center (centromere)
    x0, y0 = x0y0

    # calculate half width
    hw = 0.5*width

    dtelo = 0.5*width
    dxcent = 0.2*width
    dycent = 0.5*width

    # Prepare the data for the PathPatch below.
    Path = mpath.Path
    codes, verts = zip(*[
        # p-arm up:
        (Path.MOVETO, [x0-hw, y0+dycent]),
        (Path.LINETO, [x0-hw, y0+parm-dtelo]),
        # p-telomere curve:
        (Path.CURVE3, [x0-hw, y0+parm]),
        (Path.CURVE3, [x0, y0+parm]),
        (Path.CURVE3, [x0+hw, y0+parm]),
        (Path.CURVE3, [x0+hw, y0+parm-dtelo]),
        # p-arm down:
        (Path.LINETO, [x0+hw, y0+dycent]),
        # centromere S curve (right)
        # (1S)
        (Path.CURVE4, [x0+hw, y0+0.5*dycent]),
        (Path.CURVE4, [x0+hw-dxcent, y0+0.5*dycent]),
        (Path.CURVE4, [x0+hw-dxcent, y0]),
        # (2S)
        (Path.CURVE4, [x0+hw-dxcent, y0-0.5*dycent]),
        (Path.CURVE4, [x0+hw, y0-0.5*dycent]),
        (Path.CURVE4, [x0+hw, y0-dycent]),
        # q-arm down
        (Path.LINETO, [x0+hw, y0-qarm+dtelo]),
        # q-telomere curve:
        (Path.CURVE3, [x0+hw, y0-qarm]),
        (Path.CURVE3, [x0, y0-qarm]),
        (Path.CURVE3, [x0-hw, y0-qarm]),
        (Path.CURVE3, [x0-hw, y0-qarm+dtelo]),
        # q-arm up
        (Path.LINETO, [x0-hw, y0-dycent]),
        # centromere S curve (left)
        (Path.CURVE4, [x0-hw, y0-0.5*dycent]),
        (Path.CURVE4, [x0-hw+dxcent, y0-0.5*dycent]),
        (Path.CURVE4, [x0-hw+dxcent, y0]),
        (Path.CURVE4, [x0-hw+dxcent, y0+0.5*dycent]),
        (Path.CURVE4, [x0-hw, y0+0.5*dycent]),
        (Path.CURVE4, [x0-hw, y0+dycent]),
        (Path.CLOSEPOLY, [x0-hw, y0+dycent]),
    ])

    verts = Affine2D().rotate_deg_around(x0, y0, rotation_deg).transform(verts)
    chrom = mpatches.PathPatch(mpath.Path(verts, codes), ec="none")
    chrom.set(color=color)

    patches_to_draw = []
    if patches is not None:
        # go over patches ...
        for _arm, _from, _to, pcolor in patches:
            hw = hw + 0.005
            if _arm == "q":
                codes, verts = zip(*[
                    # rectangle
                    (Path.MOVETO, [x0-hw, y0-_from]),
                    (Path.LINETO, [x0-hw, y0-_to]),
                    (Path.LINETO, [x0+hw, y0-_to]),
                    (Path.LINETO, [x0+hw, y0-_from]),
                    (Path.CLOSEPOLY, [x0-hw, y0-_from]),
                ])
            elif _arm == "p":
                codes, verts = zip(*[
                    # rectangle
                    (Path.MOVETO, [x0-hw, y0+_from]),
                    (Path.LINETO, [x0-hw, y0+_to]),
                    (Path.LINETO, [x0+hw, y0+_to]),
                    (Path.LINETO, [x0+hw, y0+_from]),
                    (Path.CLOSEPOLY, [x0-hw, y0+_from]),
                ])
            else:
                raise("arms !!!")
            verts = Affine2D().rotate_deg_around(x0, y0, rotation_deg).transform(verts)
            part = mpatches.PathPatch(mpath.Path(verts, codes), ec="none")
            part.set(color=pcolor)
            patches_to_draw.append(part)

    return chrom, patches_to_draw




# draw 2 arm chromosome ....
def get_chrom_path_highlight(
    x0y0,  # position of the centromere ...
    parm,  # p arm length
    qarm,  # q arm length
    width,  # width
    color="grey",
    pshade=0,  # fraction of p arm to be highlighted
    qshade=0,  # fraction of q arm to be highlighted
    colorp="black",
    colorq="black",
):
    """
    a function that also highlight terminal part of
    p and q arms ...
    """
    # unpack coordinates of the center (centromere)
    x0, y0 = x0y0

    # calculate half width
    hw = 0.5*width

    dtelo = 0.5*width
    dxcent = 0.2*width
    dycent = 0.5*width

    # Prepare the data for the PathPatch below.
    Path = mpath.Path
    codes, verts = zip(*[
        # p-arm up:
        (Path.MOVETO, [x0-hw, y0+dycent]),
        (Path.LINETO, [x0-hw, y0+parm-dtelo]),
        # p-telomere curve:
        (Path.CURVE3, [x0-hw, y0+parm]),
        (Path.CURVE3, [x0, y0+parm]),
        (Path.CURVE3, [x0+hw, y0+parm]),
        (Path.CURVE3, [x0+hw, y0+parm-dtelo]),
        # p-arm down:
        (Path.LINETO, [x0+hw, y0+dycent]),
        # centromere S curve (right)
        # (1S)
        (Path.CURVE4, [x0+hw, y0+0.5*dycent]),
        (Path.CURVE4, [x0+hw-dxcent, y0+0.5*dycent]),
        (Path.CURVE4, [x0+hw-dxcent, y0]),
        # (2S)
        (Path.CURVE4, [x0+hw-dxcent, y0-0.5*dycent]),
        (Path.CURVE4, [x0+hw, y0-0.5*dycent]),
        (Path.CURVE4, [x0+hw, y0-dycent]),
        # q-arm down
        (Path.LINETO, [x0+hw, y0-qarm+dtelo]),
        # q-telomere curve:
        (Path.CURVE3, [x0+hw, y0-qarm]),
        (Path.CURVE3, [x0, y0-qarm]),
        (Path.CURVE3, [x0-hw, y0-qarm]),
        (Path.CURVE3, [x0-hw, y0-qarm+dtelo]),
        # q-arm up
        (Path.LINETO, [x0-hw, y0-dycent]),
        # centromere S curve (left)
        (Path.CURVE4, [x0-hw, y0-0.5*dycent]),
        (Path.CURVE4, [x0-hw+dxcent, y0-0.5*dycent]),
        (Path.CURVE4, [x0-hw+dxcent, y0]),
        (Path.CURVE4, [x0-hw+dxcent, y0+0.5*dycent]),
        (Path.CURVE4, [x0-hw, y0+0.5*dycent]),
        (Path.CURVE4, [x0-hw, y0+dycent]),
        (Path.CLOSEPOLY, [x0-hw, y0+dycent]),
    ])
    chrom = mpatches.PathPatch(mpath.Path(verts, codes), ec="none")
    chrom.set(color=color)

    hw = hw + 0.005
    if pshade > 0:
        dy_pshade = (1.0-pshade)*parm
        if dy_pshade < dycent:
            dy_pshade = dycent
        # p -arm highlight
        codes, verts = zip(*[
            # p-arm up:
            (Path.MOVETO, [x0-hw, y0+dy_pshade]),
            (Path.LINETO, [x0-hw, y0+parm-dtelo]),
            # p-telomere curve:
            (Path.CURVE3, [x0-hw, y0+parm]),
            (Path.CURVE3, [x0, y0+parm]),
            (Path.CURVE3, [x0+hw, y0+parm]),
            (Path.CURVE3, [x0+hw, y0+parm-dtelo]),
            # p-arm down:
            (Path.LINETO, [x0+hw, y0+dy_pshade]),
            (Path.CLOSEPOLY, [x0-hw, y0+dy_pshade]),
        ])
        part = mpatches.PathPatch(mpath.Path(verts, codes), ec="none")
        part.set(color=colorp)
    else:
        part = None
        dy_pshade = 0

    if qshade > 0:
        # Prepare the data for the PathPatch below.
        dy_qshade = (1.0-qshade)*qarm
        if dy_qshade < dycent:
            dy_qshade = dycent
        codes, verts = zip(*[
            (Path.MOVETO, [x0+hw, y0-dy_qshade]),
            # q-arm down
            (Path.LINETO, [x0+hw, y0-qarm+dtelo]),
            # q-telomere curve:
            (Path.CURVE3, [x0+hw, y0-qarm]),
            (Path.CURVE3, [x0, y0-qarm]),
            (Path.CURVE3, [x0-hw, y0-qarm]),
            (Path.CURVE3, [x0-hw, y0-qarm+dtelo]),
            # q-arm up
            (Path.LINETO, [x0-hw, y0-dy_qshade]),
            (Path.CLOSEPOLY, [x0+hw, y0-dy_qshade]),
        ])
        qart = mpatches.PathPatch(mpath.Path(verts, codes), ec="none")
        qart.set(color=colorq)
    else:
        qart = None
        dy_qshade = 0

    return chrom, (part, y0+dy_pshade), (qart, y0-dy_qshade)


# Extended Figure 2 - Nup93 heatmap ...

In [ ]:
margin = 0.2
matw = 1.333
cbarh = 0.07
cbarw = 0.7*matw

patch_kwargs = dict(color="black", fc="none", lw=0.5)

_mev_color = "tab:blue"
_pev_color = "tab:red"

# # _msample = "m5hR1R2"
# # _psample = "p5hR1R2"
# _msample = 'N93m5'
# _psample = 'N93p5'

for _fig_fname, (_msample, _psample) in {
    "Fig2A.svg": ("m5hR1R2", "p5hR1R2"),
    "FigE2A.svg": ("N93m5", "N93p5"),
}.items():
    _mev250_fname = f"ev_bigwig/{_msample}.250kb.bw"
    _pev250_fname = f"ev_bigwig/{_psample}.250kb.bw"
    _mev50_fname = f"ev_bigwig/{_msample}.50kb.bw"
    _pev50_fname = f"ev_bigwig/{_psample}.50kb.bw"
    _mev10_fname = f"ev_bigwig/{_msample}.10kb.bw"
    _pev10_fname = f"ev_bigwig/{_psample}.10kb.bw"
    _region12_factor = (region2_10[2] - region2_10[1])/(region1_10[2] - region1_10[1])
    # The first items are for padding and the second items are for the axes, sizes are in inch.
    h = [
        Size.Fixed(margin), Size.Fixed(0.125*matw),
        Size.Fixed(0.4*margin), Size.Fixed(matw),
        Size.Fixed(margin), Size.Fixed(matw),
        Size.Fixed(margin), Size.Fixed(matw),
        Size.Fixed(0.2*margin), Size.Fixed(matw*_region12_factor),
    ]
    # goes from bottom to the top ...
    v = [
        Size.Fixed(0.1), Size.Fixed(cbarh),
        Size.Fixed(0.2*margin), Size.Fixed(matw),
        Size.Fixed(0.2*margin), Size.Fixed(matw*0.2),
        Size.Fixed(0.2*margin), Size.Fixed(matw),
    ]
    # set figsize based on the tiling provided ...
    fig_width = sum(_h.fixed_size for _h in h)
    fig_height = sum(_v.fixed_size for _v in v)
    fig = plt.figure(
        figsize=(fig_width, fig_height),
        # facecolor='lightblue'
    )
    print(f"figure overall is {fig_width=} {fig_height=}")

    divider = Divider(fig, (0, 0, 1, 1), h, v, aspect=False)
    axs = {}
    # chrom schematics
    axs["chrom1"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=1, ny=1+2))
    axs["chrom2"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=1, ny=5+2))
    # 250 kb one ... - region1 and 2 combined into 1
    axs["hmap1_reg1_bs250"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=1+2, ny=1+2))
    axs["track_reg1_bs250"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=1+2, ny=3+2))
    axs["hmap2_reg1_bs250"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=1+2, ny=5+2))
    # 50 kb ...
    axs["hmap1_reg1_bs50"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=3+2, ny=1+2))
    axs["track_reg1_bs50"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=3+2, ny=3+2))
    axs["hmap2_reg1_bs50"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=3+2, ny=5+2))
    # # 10 kb region1 and region2 ....
    axs["hmap1_reg1_bs10"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=5+2, ny=1+2))
    axs["track_reg1_bs10"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=5+2, ny=3+2))
    axs["hmap2_reg1_bs10"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=5+2, ny=5+2))

    axs["hmap1_reg2_bs10"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=7+2, ny=1+2))
    axs["track_reg2_bs10"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=7+2, ny=3+2))
    axs["hmap2_reg2_bs10"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=7+2, ny=5+2))

    # define "prime" Sizes just to make way for fancy colorbarts ...
    h_prime = [
        Size.Fixed(margin), Size.Fixed(0.125*matw),
        Size.Fixed(0.4*margin), Size.Fixed(matw - cbarw), Size.Fixed(cbarw),
        Size.Fixed(margin), Size.Fixed(matw - cbarw), Size.Fixed(cbarw),
        Size.Fixed(margin), Size.Fixed(matw),
        Size.Fixed(0.2*margin), Size.Fixed(matw*_region12_factor - cbarw), Size.Fixed(cbarw),
    ]
    divider_prime = Divider(fig, (0, 0, 1, 1), h_prime, v, aspect=False)
    # colorbar axis :
    axs["cbar_bs250"] = fig.add_axes(divider_prime.get_position(), axes_locator=divider_prime.new_locator(nx=2+2, ny=1))
    axs["cbar_bs50"] = fig.add_axes(divider_prime.get_position(), axes_locator=divider_prime.new_locator(nx=5+2, ny=1))
    axs["cbar_bs10"] = fig.add_axes(divider_prime.get_position(), axes_locator=divider_prime.new_locator(nx=10+2, ny=1))

    ########################################
    for k, ax in axs.items():
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)

    #################################
    # chrom schematics
    _6pl = arms_dict["chr6_p"]
    _6ql = arms_dict["chr6_q"]
    _7pl = arms_dict["chr7_p"]
    _7ql = arms_dict["chr7_q"]
    _ww = 0.1*(_6pl+_6ql+_7pl+_7ql)

    _6qs = np.diff(region1_250[1:]).item()//1000000
    _7ps = np.diff(region2_250[1:]).item()//1000000

    _6, _ , (_6h, _6h_dy) = get_chrom_path_highlight(
        (0, _6ql+0.11*_ww),  # position of the centromere ...
        parm=_6pl,  # p arm length
        qarm=_6ql,  # q arm length
        width=_ww,  # width
        color="lightsteelblue",
        qshade=_6qs/_6ql,  # fraction of q arm to be highlighted
        colorq="dimgray",
    )
    _7, (_7h, _7h_dy) , _ = get_chrom_path_highlight(
        (0, -_7pl-0.11*_ww),  # position of the centromere ...
        parm=_7pl,  # p arm length
        qarm=_7ql,  # q arm length
        width=_ww,  # width
        color="lightsteelblue",
        pshade=(_7ps-2)/_7pl,  # fraction of p arm to be highlighted
        colorp="dimgray",
    )
    axs["chrom1"].set_axis_off()
    axs["chrom1"].add_artist(_6)
    axs["chrom1"].add_artist(_7)
    axs["chrom1"].add_artist(_6h)
    axs["chrom1"].add_artist(_7h)
    axs["chrom1"].set_xlim(-1.0*_ww, 1.0*_ww)
    axs["chrom1"].set_ylim(-(_7pl+_7ql)-(0.11+0.1)*_ww, (_6pl+_6ql)+(0.11+0.1)*_ww)

    con = ConnectionPatch(
        xyA=(0.5*_ww, _6h_dy),
        coordsA=axs["chrom1"].transData,
        xyB=(0.0, 1.0),
        coordsB=axs["hmap1_reg1_bs250"].transAxes,
        **patch_kwargs,
    )
    axs["hmap1_reg1_bs250"].add_artist(con)

    con = ConnectionPatch(
        xyA=(0.5*_ww, _7h_dy),
        coordsA=axs["chrom1"].transData,
        xyB=(0.0, 0.0),
        coordsB=axs["hmap1_reg1_bs250"].transAxes,
        **patch_kwargs,
    )
    axs["hmap1_reg1_bs250"].add_artist(con)
    ###########################################################################

    ###########################################################################
    _6, _ , (_6h, _6h_dy) = get_chrom_path_highlight(
        (0, _6ql+0.11*_ww),  # position of the centromere ...
        parm=_6pl,  # p arm length
        qarm=_6ql,  # q arm length
        width=_ww,  # width
        color="lightsteelblue",
        qshade=_6qs/_6ql,  # fraction of q arm to be highlighted
        colorq="dimgray",
    )
    _7, (_7h, _7h_dy) , _ = get_chrom_path_highlight(
        (0, -_7pl-0.11*_ww),  # position of the centromere ...
        parm=_7pl,  # p arm length
        qarm=_7ql,  # q arm length
        width=_ww,  # width
        color="lightsteelblue",
        pshade=(_7ps-2)/_7pl,  # fraction of p arm to be highlighted
        colorp="dimgray",
    )
    axs["chrom2"].set_axis_off()
    axs["chrom2"].add_artist(_6)
    axs["chrom2"].add_artist(_7)
    axs["chrom2"].add_artist(_6h)
    axs["chrom2"].add_artist(_7h)
    axs["chrom2"].set_xlim(-1.0*_ww, 1.0*_ww)
    axs["chrom2"].set_ylim(-(_7pl+_7ql)-(0.11+0.1)*_ww, (_6pl+_6ql)+(0.11+0.1)*_ww)

    con = ConnectionPatch(
        xyA=(0.5*_ww, _6h_dy),
        coordsA=axs["chrom2"].transData,
        xyB=(0.0, 1.0),
        coordsB=axs["hmap2_reg1_bs250"].transAxes,
        **patch_kwargs,
    )
    axs["hmap2_reg1_bs250"].add_artist(con)

    con = ConnectionPatch(
        xyA=(0.5*_ww, _7h_dy),
        coordsA=axs["chrom2"].transData,
        xyB=(0.0, 0.0),
        coordsB=axs["hmap2_reg1_bs250"].transAxes,
        **patch_kwargs,
    )
    axs["hmap2_reg1_bs250"].add_artist(con)
    ###########################################################################

    ##################################
    clr250 = telo_clrs250[_msample]
    clr50 = telo_clrs50[_msample]
    clr10 = telo_clrs10[_msample]

    # heatmap and track for 250 kb
    _i1_250, _i10_250 = clr250.extent(region1_250)
    _i20_250, _i2_250 = clr250.extent(region2_250)
    _binning_factor_250 = 1
    _bins1 = int(_binning_factor_250*(_i10_250-_i1_250))
    _bins2 = int(_binning_factor_250*(_i2_250-_i20_250))
    # _track_size_250 = _i2_250 - _i1_250
    _track_size_250 = _bins1 + _bins2
    mtrack1_250 = np.nan_to_num(bbi.fetch(_mev250_fname,region1_250[0],region1_250[1],region1_250[2], bins=_bins1, missing=.0, oob=.0 ))
    mtrack2_250 = np.nan_to_num(bbi.fetch(_mev250_fname,region2_250[0],region2_250[1],region2_250[2], bins=_bins2, missing=.0, oob=.0 ))
    ptrack1_250 = np.nan_to_num(bbi.fetch(_pev250_fname,region1_250[0],region1_250[1],region1_250[2], bins=_bins1, missing=.0, oob=.0 ))
    ptrack2_250 = np.nan_to_num(bbi.fetch(_pev250_fname,region2_250[0],region2_250[1],region2_250[2], bins=_bins2, missing=.0, oob=.0 ))

    # heatmap and track @ 50 kb ...
    _i1_50, _i2_50 = clr50.extent(region1_50)
    _binning_factor_50 = 1
    _track_size_50 = int(_binning_factor_50*(_i2_50 - _i1_50))
    mtrack1_50 = np.nan_to_num(bbi.fetch(_mev50_fname,region1_50[0],region1_50[1],region1_50[2], bins=_track_size_50 ))
    ptrack1_50 = np.nan_to_num(bbi.fetch(_pev50_fname,region1_50[0],region1_50[1],region1_50[2], bins=_track_size_50 ))

    # and region1 and 2 @ 10kb ...
    _i1_10, _i2_10 = clr10.extent(region1_10)
    _j1_10, _j2_10 = clr10.extent(region2_10)
    _track_size1_10 = _i2_10 - _i1_10
    _track_size2_10 = _j2_10 - _j1_10
    mtrack1_10 = np.nan_to_num(bbi.fetch(_mev10_fname,region1_10[0],region1_10[1],region1_10[2], bins=_i2_10-_i1_10 ))
    mtrack2_10 = np.nan_to_num(bbi.fetch(_mev10_fname,region2_10[0],region2_10[1],region2_10[2], bins=_j2_10-_j1_10 ))
    ptrack1_10 = np.nan_to_num(bbi.fetch(_pev10_fname,region1_10[0],region1_10[1],region1_10[2], bins=_i2_10-_i1_10 ))
    ptrack2_10 = np.nan_to_num(bbi.fetch(_pev10_fname,region2_10[0],region2_10[1],region2_10[2], bins=_j2_10-_j1_10 ))

    for ax_tag, ax in axs.items():
        if "track" in ax_tag:
            ax.axhline(0, color="grey", lw=0.5)
            ax.set_ylim(-2, 2)

    # 250 draw ...
    hm, hmkwargs = get_hm(clr250, _i1_250, _i2_250, **vlims_dict["250"])
    axs["hmap2_reg1_bs250"].imshow(hm, **hmkwargs)
    print(f"250kb heatmap is {_track_size_250} worth of pixels")
    ax = axs["track_reg1_bs250"]
    ax.plot(np.r_[mtrack1_250 ,mtrack2_250], lw=0.5, color=_mev_color)
    ax.plot(np.r_[ptrack1_250, ptrack2_250], lw=0.5, color=_pev_color)
    ax.set_xlim(0, _track_size_250)
    # 50 draw ...
    hm, hmkwargs = get_hm(clr50, _i1_50, _i2_50, **vlims_dict["50"])
    axs["hmap2_reg1_bs50"].imshow(hm, **hmkwargs)
    print(f"50kb heatmap is {_track_size_50} worth of pixels")
    ax = axs["track_reg1_bs50"]
    ax.plot(mtrack1_50, lw=0.5, color=_mev_color)
    ax.plot(ptrack1_50, lw=0.5, color=_pev_color)
    ax.set_xlim(0, _track_size_50)
    # 10 ...
    hm, hmkwargs = get_hm(clr10, _i1_10, _i2_10, **vlims_dict["10_1"])
    axs["hmap2_reg1_bs10"].imshow(hm, **hmkwargs)
    print(f"1st 510kb heatmap is {_track_size1_10} worth of pixels")
    ax = axs["track_reg1_bs10"]
    ax.plot(mtrack1_10, lw=0.5, color=_mev_color)
    ax.plot(ptrack1_10, lw=0.5, color=_pev_color)
    ax.set_xlim(0, _track_size1_10)
    # and region2 @ 10kb ...
    hm, hmkwargs = get_hm(clr10, _i1_10, _i2_10, _j1_10, _j2_10, **vlims_dict["10_2"])
    axs["hmap2_reg2_bs10"].imshow(hm, **hmkwargs)
    print(f"2nd 10kb heatmap is {_track_size2_10} worth of pixels")
    ax = axs["track_reg2_bs10"]
    ax.plot(mtrack2_10, lw=0.5, color=_mev_color)
    ax.plot(ptrack2_10, lw=0.5, color=_pev_color)
    ax.set_xlim(0, _track_size2_10)

    # inset on 250 - connection to 50 ....
    _inset_from, _inset_to = clr250.extent(region1_50)
    _inset_size = _inset_to - _inset_from
    _inset_from -= _i1_250

    #  Now the depletion ...
    clr250 = telo_clrs250[_psample]
    clr50 = telo_clrs50[_psample]
    clr10 = telo_clrs10[_psample]
    # 250 draw ...
    hm, hmkwargs = get_hm(clr250, _i1_250, _i2_250, **vlims_dict["250"])
    axs["hmap1_reg1_bs250"].imshow(hm, **hmkwargs)
    # 50 draw ...
    hm, hmkwargs = get_hm(clr50, _i1_50, _i2_50, **vlims_dict["50"])
    axs["hmap1_reg1_bs50"].imshow(hm, **hmkwargs)
    # 10 ...
    hm, hmkwargs = get_hm(clr10, _i1_10, _i2_10, **vlims_dict["10_1"])
    axs["hmap1_reg1_bs10"].imshow(hm, **hmkwargs)
    # and region2 @ 10kb ...
    hm, hmkwargs = get_hm(clr10, _i1_10, _i2_10, _j1_10, _j2_10, **vlims_dict["10_2"])
    axs["hmap1_reg2_bs10"].imshow(hm, **hmkwargs)

    # inset on 250 - connection to 50 ....
    _inset_from, _inset_to = clr250.extent(region1_50)
    _inset_size = _inset_to - _inset_from
    _inset_from -= _i1_250

    _rec = Rectangle(
        (_inset_from, _inset_from),
        _inset_size, _inset_size,
        **patch_kwargs,
    )
    # draw lines connecting inset and axes ...
    line250_50_upper = BboxConnector(
        bbox1=_rec, bbox2=axs["hmap1_reg1_bs50"].bbox,
        loc1=1, loc2=2,
        **patch_kwargs,
    )
    line250_50_lower = BboxConnector(
        bbox1=_rec, bbox2=axs["hmap1_reg1_bs50"].bbox,
        loc1=4, loc2=3,
        **patch_kwargs,
    )
    # draw those box and lines ...
    axs["hmap1_reg1_bs250"].add_patch(_rec)
    fig.add_artist(line250_50_upper)
    fig.add_artist(line250_50_lower)

    # inset boxes and connections between 50kb and 10kb ...
    _inset1_from, _inset1_to = clr50.extent(region1_10)
    _inset2_from, _inset2_to = clr50.extent(region2_10)
    _inset1_size = _inset1_to - _inset1_from
    _inset2_size = _inset2_to - _inset2_from
    # bigger box around both of these ...
    _inset12_size = _inset2_to - _inset1_from
    # adjust to the heatmap they're in ...
    _inset1_from -= _i1_50
    _inset2_from -= _i1_50
    _extra = 8  # big box adjustment ...

    _rec1 = Rectangle(
        (_inset1_from, _inset1_from),
        _inset1_size, _inset1_size,
        **patch_kwargs,
    )
    _rec2 = Rectangle(
        (_inset2_from, _inset1_from),
        _inset2_size, _inset1_size,
        **patch_kwargs,
    )
    # bigger box around both of those ...
    _rec12 = Rectangle(
        (_inset1_from-_extra, _inset1_from-_extra),
        _inset12_size+2*_extra, _inset1_size+2*_extra,
        **patch_kwargs,
    )
    # draw lines connecting inset and axes ...
    line50_10_upper = BboxConnector(
        bbox1=_rec12, bbox2=axs["hmap1_reg1_bs10"].bbox,
        loc1=1, loc2=2,
        **patch_kwargs,
    )
    line50_10_lower = BboxConnector(
        bbox1=_rec12, bbox2=axs["hmap1_reg1_bs10"].bbox,
        loc1=4, loc2=3,
        **patch_kwargs,
    )
    # draw it ...
    axs["hmap1_reg1_bs50"].add_patch(_rec1)
    axs["hmap1_reg1_bs50"].add_patch(_rec2)
    axs["hmap1_reg1_bs50"].add_patch(_rec12)
    fig.add_artist(line50_10_upper)
    fig.add_artist(line50_10_lower)

    # same for the upper row of axes ...
    #####################################################
    _rec = Rectangle(
        (_inset_from, _inset_from),
        _inset_size, _inset_size,
        **patch_kwargs,
    )
    # draw lines connecting inset and axes ...
    line250_50_upper = BboxConnector(
        bbox1=_rec, bbox2=axs["hmap2_reg1_bs50"].bbox,
        loc1=1, loc2=2,
        **patch_kwargs,
    )
    line250_50_lower = BboxConnector(
        bbox1=_rec, bbox2=axs["hmap2_reg1_bs50"].bbox,
        loc1=4, loc2=3,
        **patch_kwargs,
    )
    # draw those box and lines ...
    axs["hmap2_reg1_bs250"].add_patch(_rec)
    fig.add_artist(line250_50_upper)
    fig.add_artist(line250_50_lower)

    _rec1 = Rectangle(
        (_inset1_from, _inset1_from),
        _inset1_size, _inset1_size,
        **patch_kwargs,
    )
    _rec2 = Rectangle(
        (_inset2_from, _inset1_from),
        _inset2_size, _inset1_size,
        **patch_kwargs,
    )
    # bigger box around both of those ...
    _rec12 = Rectangle(
        (_inset1_from-_extra, _inset1_from-_extra),
        _inset12_size+2*_extra, _inset1_size+2*_extra,
        **patch_kwargs,
    )
    # draw lines connecting inset and axes ...
    line50_10_upper = BboxConnector(
        bbox1=_rec12, bbox2=axs["hmap2_reg1_bs10"].bbox,
        loc1=1, loc2=2,
        **patch_kwargs,
    )
    line50_10_lower = BboxConnector(
        bbox1=_rec12, bbox2=axs["hmap2_reg1_bs10"].bbox,
        loc1=4, loc2=3,
        **patch_kwargs,
    )
    # draw it ...
    axs["hmap2_reg1_bs50"].add_patch(_rec1)
    axs["hmap2_reg1_bs50"].add_patch(_rec2)
    axs["hmap2_reg1_bs50"].add_patch(_rec12)
    fig.add_artist(line50_10_upper)
    fig.add_artist(line50_10_lower)

    # 250
    _cax = axs["cbar_bs250"]
    _vmin, _vmax = vlims_dict["250"]["vmin"], vlims_dict["250"]["vmax"]

    plt.colorbar(
        # cm.ScalarMappable(norm=PowerNorm(1/3., vmin=_vmin, vmax=_vmax), cmap=mycmap),
        cm.ScalarMappable(norm=LogNorm(vmin=_vmin, vmax=_vmax), cmap=mycmap),
        cax=_cax,
        orientation="horizontal",
    )

    _cax.xaxis.set_visible(True)
    _ticks_power = range(
        np.ceil(np.log10(_vmin)).astype(int),
        np.floor(np.log10(_vmax)).astype(int)+1,
    )
    _cax.set_xticks([np.power(10.,_p) for _p in _ticks_power])
    _cax.set_xticklabels(['$\mathdefault{10^{%i}}$' % l for l in _ticks_power])
    _cax.xaxis.set_tick_params(labelsize=6, pad=0.1, length=2)
    _cax.minorticks_off()


    # 50
    _cax = axs["cbar_bs50"]
    _vmin, _vmax = vlims_dict["50"]["vmin"], vlims_dict["50"]["vmax"]

    plt.colorbar(
        # cm.ScalarMappable(norm=PowerNorm(1/3., vmin=_vmin, vmax=_vmax), cmap=mycmap),
        cm.ScalarMappable(norm=LogNorm(vmin=_vmin, vmax=_vmax), cmap=mycmap),
        cax=_cax,
        orientation="horizontal",
    )

    _cax.xaxis.set_visible(True)
    _ticks_power = range(
        np.ceil(np.log10(_vmin)).astype(int),
        np.floor(np.log10(_vmax)).astype(int)+1,
    )
    _cax.set_xticks([np.power(10.,_p) for _p in _ticks_power])
    _cax.set_xticklabels(['$\mathdefault{10^{%i}}$' % l for l in _ticks_power])
    _cax.xaxis.set_tick_params(labelsize=6, pad=0.1, length=2)
    _cax.minorticks_off()


    # 10
    _cax = axs["cbar_bs10"]
    _vmin, _vmax = vlims_dict["10_2"]["vmin"], vlims_dict["10_2"]["vmax"]

    plt.colorbar(
        # cm.ScalarMappable(norm=PowerNorm(1/3., vmin=_vmin, vmax=_vmax), cmap=mycmap),
        cm.ScalarMappable(norm=LogNorm(vmin=_vmin, vmax=_vmax), cmap=mycmap),
        cax=_cax,
        orientation="horizontal",
    )

    _cax.xaxis.set_visible(True)
    _ticks_power = range(
        np.ceil(np.log10(_vmin)).astype(int),
        np.floor(np.log10(_vmax)).astype(int)+1,
    )
    _cax.set_xticks([np.power(10.,_p) for _p in _ticks_power])
    _cax.set_xticklabels(['$\mathdefault{10^{%i}}$' % l for l in _ticks_power])
    _cax.xaxis.set_tick_params(labelsize=6, pad=0.1, length=2)
    _cax.minorticks_off()

    plt.savefig(_fig_fname, dpi=300)

## do similar figure for the extended MP figure ...

In [ ]:
# adjust colobars for the mp ...
vlims_dict["10_1"]["vmin"] = 0.00005
vlims_dict["10_2"]["vmin"] = 0.00005

vlims_dict["10_1"]["vmax"] = 0.0075
vlims_dict["10_2"]["vmax"] = 0.0075

In [ ]:
sample_m = "m10hR1R2"
sample_mp = "mp10hR1R2"
sample_p = "p10hR1R2"

_region12_factor = (region2_10[2] - region2_10[1])/(region1_10[2] - region1_10[1])
# The first items are for padding and the second items are for the axes, sizes are in inch.
h = [
    Size.Fixed(margin), Size.Fixed(0.125*matw),
    Size.Fixed(0.4*margin), Size.Fixed(matw),
    Size.Fixed(0.2*margin), Size.Fixed(matw*_region12_factor),
    Size.Fixed(0.2*margin), Size.Fixed(cbarh),
    Size.Fixed(margin),
]
# goes from bottom to the top ...
v = [
    Size.Fixed(margin), Size.Fixed(matw*0.2),
    Size.Fixed(0.2*margin), Size.Fixed(matw),
    Size.Fixed(0.2*margin), Size.Fixed(matw),
    Size.Fixed(0.2*margin), Size.Fixed(matw),
    Size.Fixed(0.9*margin), Size.Fixed(0.125*matw),
    Size.Fixed(margin),
]
# set figsize based on the tiling provided ...
fig_width = sum(_h.fixed_size for _h in h)
fig_height = sum(_v.fixed_size for _v in v)
fig = plt.figure(
    figsize=(fig_width, fig_height),
    # facecolor='lightblue'
)
print(f"figure overall is {fig_width=} {fig_height=}")
divider = Divider(fig, (0, 0, 1, 1), h, v, aspect=False)


axs = {}
# chrom schematics
axs["chrom_m"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=1, ny=7))
axs["chrom_p"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=1, ny=5))
axs["chrom_mp"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=1, ny=3))
axs["chrom"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=3, nx1=6, ny=9))
# # 10 kb region1 and region2 ....
axs["hmap_m_reg1_bs10"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=3, ny=7))
axs["hmap_p_reg1_bs10"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=3, ny=5))
axs["hmap_mp_reg1_bs10"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=3, ny=3))

axs["hmap_m_reg2_bs10"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=5, ny=7))
axs["hmap_p_reg2_bs10"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=5, ny=5))
axs["hmap_mp_reg2_bs10"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=5, ny=3))

axs["track_reg1_bs10"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=3, ny=1))
axs["track_reg2_bs10"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=5, ny=1))

cbar_ax = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=7, ny=7))

########################################
for k, ax in axs.items():
    # ax.xaxis.set_visible(False)
    # ax.yaxis.set_visible(True)
    ax.set_xticks([])
    ax.set_yticks([])
# cbar_ax.xaxis.set_visible(False)
# cbar_ax.yaxis.set_visible(False)
cbar_ax.xaxis.set_visible(False)
cbar_ax.yaxis.set_visible(False)

# #################################
# chrom schematics
_6pl = arms_dict["chr6_p"]
_6ql = arms_dict["chr6_q"]
_ww = 0.1*(_6pl+_6ql)
_region_patches = [
    ("q", np.floor(region1_10[1]/1000000)-_6pl, np.ceil(region1_10[2]/1000000)-_6pl, "dimgray",),
    ("q", np.floor(region2_10[1]/1000000)-_6pl, np.ceil(region2_10[2]/1000000)-_6pl, "dimgray",),
]

for ax in [axs["chrom_m"], axs["chrom_p"], axs["chrom_mp"]]:
    _6, _rps = get_chrom_path_wpathes(
        (0, 0),  # position of the centromere ...
        parm=_6pl,  # p arm length
        qarm=_6ql,  # q arm length
        width=_ww,  # width
        color="lightsteelblue",
        patches=_region_patches,
    )
    for key, spine in ax.spines.items():
        spine.set_visible(False)
    # ax.set_axis_off()
    ax.add_artist(_6)
    for _p in _rps:
        ax.add_artist(_p)
    ax.set_xlim(-1.0*_ww, 1.0*_ww)
    ax.set_ylim(-_6ql-(0.11+0.1)*_ww, _6pl+(0.11+0.1)*_ww)

for ax, _sample in zip(
    [axs["chrom_m"], axs["chrom_p"], axs["chrom_mp"]],
    [sample_m, sample_p, sample_mp]
):
    ax.set_ylabel(_sample, fontsize=6, labelpad=0)
    # ax.set_title(_sample, fontsize=6, pad=0)



ax = axs["chrom"]
_6, _rps = get_chrom_path_wpathes(
    (0, 0),  # position of the centromere ...
    parm=_6pl,  # p arm length
    qarm=_6ql,  # q arm length
    width=_ww,  # width
    color="lightsteelblue",
    patches=_region_patches,
    rotation_deg = 90,
)
ax.set_axis_off()
ax.add_artist(_6)
for _p in _rps:
    ax.add_patch(_p)
ax.set_ylim(-1.0*_ww, 1.0*_ww)
ax.set_xlim(-_6pl-(0.11+0.1)*_ww-4*_ww, _6ql+(0.11+0.1)*_ww+4*_ww)

con = ConnectionPatch(
    xyA=(_region_patches[0][1], -0.5*_ww-0.005),
    coordsA=ax.transData,
    xyB=(0.0, 1.0),
    coordsB=axs["hmap_m_reg1_bs10"].transAxes,
    **patch_kwargs,
)
axs["hmap_m_reg1_bs10"].add_artist(con)
con = ConnectionPatch(
    xyA=(_region_patches[0][2], -0.5*_ww-0.005),
    coordsA=ax.transData,
    xyB=(1.0, 1.0),
    coordsB=axs["hmap_m_reg1_bs10"].transAxes,
    **patch_kwargs,
)
axs["hmap_m_reg1_bs10"].add_artist(con)

con = ConnectionPatch(
    xyA=(_region_patches[1][1], -0.5*_ww-0.005),
    coordsA=ax.transData,
    xyB=(0.0, 1.0),
    coordsB=axs["hmap_m_reg2_bs10"].transAxes,
    **patch_kwargs,
)
axs["hmap_m_reg2_bs10"].add_artist(con)
con = ConnectionPatch(
    xyA=(_region_patches[1][2], -0.5*_ww-0.005),
    coordsA=ax.transData,
    xyB=(1.0, 1.0),
    coordsB=axs["hmap_m_reg2_bs10"].transAxes,
    **patch_kwargs,
)
axs["hmap_m_reg2_bs10"].add_artist(con)


# ##################################
clr10 = telo_clrs10[sample_m]
# and region1 and 2 @ 10kb ...
_i1_10, _i2_10 = clr10.extent(region1_10)
_j1_10, _j2_10 = clr10.extent(region2_10)
_track_size1_10 = _i2_10 - _i1_10
_track_size2_10 = _j2_10 - _j1_10

_track_colors = {
    sample_m : "blue",
    sample_p : "red",
    sample_mp : "purple",
}

for ax_tag, ax in axs.items():
    if "track" in ax_tag:
        ax.axhline(0, color="grey", lw=0.5)
        ax.set_ylim(-2, 2)

for _type, _sample in zip(
    ["m", "p", "mp"],
    [sample_m, sample_p, sample_mp],
):
    clr10 = telo_clrs10[_sample]
    # 10 ...
    hm, hmkwargs = get_hm(clr10, _i1_10, _i2_10, **vlims_dict["10_1"])
    hmkwargs["interpolation"] = "antialiased"
    hmkwargs["interpolation_stage"] = "data"
    axs[f"hmap_{_type}_reg1_bs10"].imshow(hm, **hmkwargs)
    print(f"1st 510kb heatmap is {_track_size1_10} worth of pixels")
    ax = axs["track_reg1_bs10"]
    _track1_10 = np.nan_to_num(bbi.fetch(f"ev_bigwig/{_sample}.10kb.bw",region1_10[0],region1_10[1],region1_10[2], bins=_track_size1_10 ))
    ax.plot(_track1_10, lw=0.5, color=_track_colors[_sample])
    ax.set_xlim(0, _track_size1_10)
    # and region2 @ 10kb ...
    hm, hmkwargs = get_hm(clr10, _i1_10, _i2_10, _j1_10, _j2_10, **vlims_dict["10_2"])
    hmkwargs["interpolation"] = "antialiased"
    hmkwargs["interpolation_stage"] = "data"
    axs[f"hmap_{_type}_reg2_bs10"].imshow(hm, **hmkwargs)
    print(f"2nd 10kb heatmap is {_track_size2_10} worth of pixels")
    ax = axs["track_reg2_bs10"]
    _track2_10 = np.nan_to_num(bbi.fetch(f"ev_bigwig/{_sample}.10kb.bw",region2_10[0],region2_10[1],region2_10[2], bins=_track_size2_10 ))
    ax.plot(_track2_10, lw=0.5, color=_track_colors[_sample])
    ax.set_xlim(0, _track_size2_10)


# single colorbar ...
_vmin, _vmax = vlims_dict["10_2"]["vmin"], vlims_dict["10_2"]["vmax"]
plt.colorbar(
    cm.ScalarMappable(norm=LogNorm(vmin=_vmin, vmax=_vmax), cmap=mycmap),
    cax=cbar_ax,
    orientation="vertical",
)

cbar_ax.yaxis.set_visible(True)
_ticks_power = range(
    np.ceil(np.log10(_vmin)).astype(int),
    np.floor(np.log10(_vmax)).astype(int)+1,
)
cbar_ax.set_yticks([np.power(10.,_p) for _p in _ticks_power])
cbar_ax.set_yticklabels(['$\mathdefault{10^{%i}}$' % l for l in _ticks_power])
cbar_ax.yaxis.set_tick_params(labelsize=6, pad=0.1, length=2)
cbar_ax.minorticks_off()

_fig_fname = "FigExt7C_mp.svg"
plt.savefig(_fig_fname, dpi=300)

```
{'250': {'vmin': 1e-05, 'vmax': 0.003},
 '50': {'vmin': 4e-05, 'vmax': 0.025},
 '10_1': {'vmin': 0.0001, 'vmax': 0.01},
 '10_2': {'vmin': 0.0001, 'vmax': 0.01}}
```

# Figure 2A in reverse order below ...

In [ ]:
# w, h = 6, 2.9
# margin = 0.2
# matw = 1.15
# cbarh = 0.07
# cbarw = 0.7*matw

# fig = plt.figure(
#     figsize=(w, h),
#     # facecolor='lightblue'
# )


# _region12_factor = (region2_10[2] - region2_10[1])/(region1_10[2] - region1_10[1])
# # The first items are for padding and the second items are for the axes, sizes are in inch.
# h = [
#     Size.Fixed(margin), Size.Fixed(matw),
#     Size.Fixed(0.2*margin), Size.Fixed(matw*_region12_factor),
#     Size.Fixed(margin), Size.Fixed(matw),
#     Size.Fixed(margin), Size.Fixed(matw),
# ]
# # goes from bottom to the top ...
# v = [
#     Size.Fixed(0.1), Size.Fixed(cbarh),
#     Size.Fixed(0.2*margin), Size.Fixed(matw),
#     Size.Fixed(0.2*margin), Size.Fixed(matw*0.2),
#     Size.Fixed(0.2*margin), Size.Fixed(matw),
# ]

# divider = Divider(fig, (0, 0, 1, 1), h, v, aspect=False)
# axs = {}
# # 250 kb one ... - region1 and 2 combined into 1
# axs["hmap1_reg1_bs250"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=7, ny=1+2))
# axs["track_reg1_bs250"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=7, ny=3+2))
# axs["hmap2_reg1_bs250"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=7, ny=5+2))
# # 50 kb ...
# axs["hmap1_reg1_bs50"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=5, ny=1+2))
# axs["track_reg1_bs50"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=5, ny=3+2))
# axs["hmap2_reg1_bs50"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=5, ny=5+2))
# # # 10 kb region1 and region2 ....
# axs["hmap1_reg1_bs10"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=1, ny=1+2))
# axs["track_reg1_bs10"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=1, ny=3+2))
# axs["hmap2_reg1_bs10"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=1, ny=5+2))

# axs["hmap1_reg2_bs10"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=3, ny=1+2))
# axs["track_reg2_bs10"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=3, ny=3+2))
# axs["hmap2_reg2_bs10"] = fig.add_axes(divider.get_position(), axes_locator=divider.new_locator(nx=3, ny=5+2))

# # define "prime" Sizes just to make way for fancy colorbarts ...
# h_prime = [
#     Size.Fixed(margin), Size.Fixed(matw),
#     Size.Fixed(0.2*margin), Size.Fixed(matw*_region12_factor - cbarw), Size.Fixed(cbarw),
#     Size.Fixed(margin), Size.Fixed(matw - cbarw), Size.Fixed(cbarw),
#     Size.Fixed(margin), Size.Fixed(matw - cbarw), Size.Fixed(cbarw),
# ]
# divider_prime = Divider(fig, (0, 0, 1, 1), h_prime, v, aspect=False)
# # colorbar axis :
# axs["cbar_bs250"] = fig.add_axes(divider_prime.get_position(), axes_locator=divider_prime.new_locator(nx=10, ny=1))
# axs["cbar_bs50"] = fig.add_axes(divider_prime.get_position(), axes_locator=divider_prime.new_locator(nx=7, ny=1))
# axs["cbar_bs10"] = fig.add_axes(divider_prime.get_position(), axes_locator=divider_prime.new_locator(nx=4, ny=1))


# ########################################
# for k, ax in axs.items():
#     ax.xaxis.set_visible(False)
#     ax.yaxis.set_visible(False)

# _msample = "m5hR1R2"
# _psample = "p5hR1R2"
# clr250 = telo_clrs250[_msample]
# clr50 = telo_clrs50[_msample]
# clr10 = telo_clrs10[_msample]

# # heatmap and track for 250 kb
# _i1_250, _i10_250 = clr250.extent(region1_250)
# _i20_250, _i2_250 = clr250.extent(region2_250)
# _track_size_250 = _i2_250 - _i1_250
# mtrack1_250 = np.nan_to_num(bbi.fetch(f"ev_bigwig/{_msample}.10kb.bw",region1_250[0],region1_250[1],region1_250[2], bins=_i10_250-_i1_250 ))
# mtrack2_250 = np.nan_to_num(bbi.fetch(f"ev_bigwig/{_msample}.10kb.bw",region2_250[0],region2_250[1],region2_250[2], bins=_i2_250-_i20_250 ))
# ptrack1_250 = np.nan_to_num(bbi.fetch(f"ev_bigwig/{_psample}.10kb.bw",region1_250[0],region1_250[1],region1_250[2], bins=_i10_250-_i1_250 ))
# ptrack2_250 = np.nan_to_num(bbi.fetch(f"ev_bigwig/{_psample}.10kb.bw",region2_250[0],region2_250[1],region2_250[2], bins=_i2_250-_i20_250 ))

# # heatmap and track @ 50 kb ...
# _i1_50, _i2_50 = clr50.extent(region1_50)
# _track_size_50 = _i2_50 - _i1_50
# mtrack1_50 = np.nan_to_num(bbi.fetch(f"ev_bigwig/{_msample}.10kb.bw",region1_50[0],region1_50[1],region1_50[2], bins=_i2_50-_i1_50 ))
# ptrack1_50 = np.nan_to_num(bbi.fetch(f"ev_bigwig/{_psample}.10kb.bw",region1_50[0],region1_50[1],region1_50[2], bins=_i2_50-_i1_50 ))

# # and region1 and 2 @ 10kb ...
# _i1_10, _i2_10 = clr10.extent(region1_10)
# _j1_10, _j2_10 = clr10.extent(region2_10)
# _track_size1_10 = _i2_10 - _i1_10
# _track_size2_10 = _j2_10 - _j1_10
# mtrack1_10 = np.nan_to_num(bbi.fetch(f"ev_bigwig/{_msample}.10kb.bw",region1_10[0],region1_10[1],region1_10[2], bins=_i2_10-_i1_10 ))
# mtrack2_10 = np.nan_to_num(bbi.fetch(f"ev_bigwig/{_msample}.10kb.bw",region2_10[0],region2_10[1],region2_10[2], bins=_j2_10-_j1_10 ))
# ptrack1_10 = np.nan_to_num(bbi.fetch(f"ev_bigwig/{_psample}.10kb.bw",region1_10[0],region1_10[1],region1_10[2], bins=_i2_10-_i1_10 ))
# ptrack2_10 = np.nan_to_num(bbi.fetch(f"ev_bigwig/{_psample}.10kb.bw",region2_10[0],region2_10[1],region2_10[2], bins=_j2_10-_j1_10 ))

# for ax_tag, ax in axs.items():
#     if "track" in ax_tag:
#         ax.axhline(0, color="grey", lw=0.5)
#         ax.set_ylim(-2, 2)

# # 250 draw ...
# hm = get_hm(clr250, _i1_250, _i2_250, **vlims_dict["250"])
# axs["hmap2_reg1_bs250"].imshow(hm)
# print(f"250kb heatmap is {_track_size_250} worth of pixels")
# ax = axs["track_reg1_bs250"]
# ax.plot(np.r_[mtrack1_250 ,mtrack2_250], lw=0.7, color="blue")
# ax.plot(np.r_[ptrack1_250, ptrack2_250], lw=0.7, color="red")
# ax.set_xlim(0, _track_size_250)
# # 50 draw ...
# hm = get_hm(clr50, _i1_50, _i2_50, **vlims_dict["50"])
# axs["hmap2_reg1_bs50"].imshow(hm)
# print(f"50kb heatmap is {_track_size_50} worth of pixels")
# ax = axs["track_reg1_bs50"]
# ax.plot(mtrack1_50, lw=0.7, color="blue")
# ax.plot(ptrack1_50, lw=0.7, color="red")
# ax.set_xlim(0, _track_size_50)
# # 10 ...
# hm = get_hm(clr10, _i1_10, _i2_10, **vlims_dict["10_1"])
# axs["hmap2_reg1_bs10"].imshow(hm)
# print(f"1st 510kb heatmap is {_track_size1_10} worth of pixels")
# ax = axs["track_reg1_bs10"]
# ax.plot(mtrack1_10, lw=0.7, color="blue")
# ax.plot(ptrack1_10, lw=0.7, color="red")
# ax.set_xlim(0, _track_size1_10)
# # and region2 @ 10kb ...
# hm = get_hm(clr10, _i1_10, _i2_10, _j1_10, _j2_10, **vlims_dict["10_2"])
# axs["hmap2_reg2_bs10"].imshow(hm)
# print(f"2nd 10kb heatmap is {_track_size2_10} worth of pixels")
# ax = axs["track_reg2_bs10"]
# ax.plot(mtrack2_10, lw=0.7, color="blue")
# ax.plot(ptrack2_10, lw=0.7, color="red")
# ax.set_xlim(0, _track_size2_10)

# # inset on 250 - connection to 50 ....
# _inset_from, _inset_to = clr250.extent(region1_50)
# _inset_size = _inset_to - _inset_from
# _inset_from -= _i1_250

# #  Now the depletion ...
# _sample = "p5hR1R2"
# clr250 = telo_clrs250[_sample]
# clr50 = telo_clrs50[_sample]
# clr10 = telo_clrs10[_sample]
# # 250 draw ...
# hm = get_hm(clr250, _i1_250, _i2_250, **vlims_dict["250"])
# axs["hmap1_reg1_bs250"].imshow(hm)
# # 50 draw ...
# hm = get_hm(clr50, _i1_50, _i2_50, **vlims_dict["50"])
# axs["hmap1_reg1_bs50"].imshow(hm)
# # 10 ...
# hm = get_hm(clr10, _i1_10, _i2_10, **vlims_dict["10_1"])
# axs["hmap1_reg1_bs10"].imshow(hm)
# # and region2 @ 10kb ...
# hm = get_hm(clr10, _i1_10, _i2_10, _j1_10, _j2_10, **vlims_dict["10_2"])
# axs["hmap1_reg2_bs10"].imshow(hm)

# # inset on 250 - connection to 50 ....
# _inset_from, _inset_to = clr250.extent(region1_50)
# _inset_size = _inset_to - _inset_from
# _inset_from -= _i1_250

# patch_kwargs = dict(color="grey", fc="none", lw=0.5)

# _rec = Rectangle(
#     (_inset_from, _inset_from),
#     _inset_size, _inset_size,
#     **patch_kwargs,
# )
# # draw lines connecting inset and axes ...
# line250_50_upper = BboxConnector(
#     bbox1=_rec, bbox2=axs["hmap1_reg1_bs50"].bbox,
#     loc1=2, loc2=1,
#     **patch_kwargs,
# )
# line250_50_lower = BboxConnector(
#     bbox1=_rec, bbox2=axs["hmap1_reg1_bs50"].bbox,
#     loc1=3, loc2=4,
#     **patch_kwargs,
# )
# # draw those box and lines ...
# axs["hmap1_reg1_bs250"].add_patch(_rec)
# fig.add_artist(line250_50_upper)
# fig.add_artist(line250_50_lower)

# # inset boxes and connections between 50kb and 10kb ...
# _inset1_from, _inset1_to = clr50.extent(region1_10)
# _inset2_from, _inset2_to = clr50.extent(region2_10)
# _inset1_size = _inset1_to - _inset1_from
# _inset2_size = _inset2_to - _inset2_from
# # bigger box around both of these ...
# _inset12_size = _inset2_to - _inset1_from
# # adjust to the heatmap they're in ...
# _inset1_from -= _i1_50
# _inset2_from -= _i1_50
# _extra = 8  # big box adjustment ...

# _rec1 = Rectangle(
#     (_inset1_from, _inset1_from),
#     _inset1_size, _inset1_size,
#     **patch_kwargs,
# )
# _rec2 = Rectangle(
#     (_inset2_from, _inset1_from),
#     _inset2_size, _inset1_size,
#     **patch_kwargs,
# )
# # bigger box around both of those ...
# _rec12 = Rectangle(
#     (_inset1_from-_extra, _inset1_from-_extra),
#     _inset12_size+2*_extra, _inset1_size+2*_extra,
#     **patch_kwargs,
# )
# # draw lines connecting inset and axes ...
# line50_10_upper = BboxConnector(
#     bbox1=_rec12, bbox2=axs["hmap1_reg2_bs10"].bbox,
#     loc1=2, loc2=1,
#     **patch_kwargs,
# )
# line50_10_lower = BboxConnector(
#     bbox1=_rec12, bbox2=axs["hmap1_reg2_bs10"].bbox,
#     loc1=3, loc2=4,
#     **patch_kwargs,
# )
# # draw it ...
# axs["hmap1_reg1_bs50"].add_patch(_rec1)
# axs["hmap1_reg1_bs50"].add_patch(_rec2)
# axs["hmap1_reg1_bs50"].add_patch(_rec12)
# fig.add_artist(line50_10_upper)
# fig.add_artist(line50_10_lower)



# # same for the upper row of axes ...
# #####################################################
# _rec = Rectangle(
#     (_inset_from, _inset_from),
#     _inset_size, _inset_size,
#     **patch_kwargs,
# )
# # draw lines connecting inset and axes ...
# line250_50_upper = BboxConnector(
#     bbox1=_rec, bbox2=axs["hmap2_reg1_bs50"].bbox,
#     loc1=2, loc2=1,
#     **patch_kwargs,
# )
# line250_50_lower = BboxConnector(
#     bbox1=_rec, bbox2=axs["hmap2_reg1_bs50"].bbox,
#     loc1=3, loc2=4,
#     **patch_kwargs,
# )
# # draw those box and lines ...
# axs["hmap2_reg1_bs250"].add_patch(_rec)
# fig.add_artist(line250_50_upper)
# fig.add_artist(line250_50_lower)

# _rec1 = Rectangle(
#     (_inset1_from, _inset1_from),
#     _inset1_size, _inset1_size,
#     **patch_kwargs,
# )
# _rec2 = Rectangle(
#     (_inset2_from, _inset1_from),
#     _inset2_size, _inset1_size,
#     **patch_kwargs,
# )
# # bigger box around both of those ...
# _rec12 = Rectangle(
#     (_inset1_from-_extra, _inset1_from-_extra),
#     _inset12_size+2*_extra, _inset1_size+2*_extra,
#     **patch_kwargs,
# )
# # draw lines connecting inset and axes ...
# line50_10_upper = BboxConnector(
#     bbox1=_rec12, bbox2=axs["hmap2_reg2_bs10"].bbox,
#     loc1=2, loc2=1,
#     **patch_kwargs,
# )
# line50_10_lower = BboxConnector(
#     bbox1=_rec12, bbox2=axs["hmap2_reg2_bs10"].bbox,
#     loc1=3, loc2=4,
#     **patch_kwargs,
# )
# # draw it ...
# axs["hmap2_reg1_bs50"].add_patch(_rec1)
# axs["hmap2_reg1_bs50"].add_patch(_rec2)
# axs["hmap2_reg1_bs50"].add_patch(_rec12)
# fig.add_artist(line50_10_upper)
# fig.add_artist(line50_10_lower)

# # fig.add_axes((0.8, 0.1, 0.3, 0.2))
# # axs["hmap2_reg1_bs250"]
# # axs["hmap2_reg1_bs50"]
# # axs["hmap2_reg1_bs10"]
# # vvv = axs["hmap2_reg2_bs10"].get_tightbbox(for_layout_only=1)


# # f = plt.figure(figsize=(0.5,2))
# # cax = f.add_subplot()


# # 250
# _cax = axs["cbar_bs250"]
# _vmin, _vmax = vlims_dict["250"]["vmin"], vlims_dict["250"]["vmax"]

# plt.colorbar(
#     cm.ScalarMappable(norm=PowerNorm(1/3., vmin=_vmin, vmax=_vmax), cmap=mycmap),
#     cax=_cax,
#     orientation="horizontal",
# )

# _cax.xaxis.set_visible(True)
# _ticks_power = range(
#     np.ceil(np.log10(_vmin)).astype(int),
#     np.floor(np.log10(_vmax)).astype(int)+1,
# )
# _cax.set_xticks([np.power(10.,_p) for _p in _ticks_power])
# _cax.set_xticklabels(['$\mathdefault{10^{%i}}$' % l for l in _ticks_power])
# # _cax.ticklabel_format(axis="x",style="sci",scilimits=(0, 0),useMathText=True)
# _cax.xaxis.set_tick_params(labelsize=6, pad=0.1, length=2)



# # 50
# _cax = axs["cbar_bs50"]
# _vmin, _vmax = vlims_dict["50"]["vmin"], vlims_dict["50"]["vmax"]

# plt.colorbar(
#     cm.ScalarMappable(norm=PowerNorm(1/3., vmin=_vmin, vmax=_vmax), cmap=mycmap),
#     cax=_cax,
#     orientation="horizontal",
# )

# _cax.xaxis.set_visible(True)
# _ticks_power = range(
#     np.ceil(np.log10(_vmin)).astype(int),
#     np.floor(np.log10(_vmax)).astype(int)+1,
# )
# _cax.set_xticks([np.power(10.,_p) for _p in _ticks_power])
# _cax.set_xticklabels(['$\mathdefault{10^{%i}}$' % l for l in _ticks_power])
# # _cax.ticklabel_format(axis="x",style="sci",scilimits=(0, 0),useMathText=True)
# _cax.xaxis.set_tick_params(labelsize=6, pad=0.1, length=2)



# # 10
# _cax = axs["cbar_bs10"]
# _vmin, _vmax = vlims_dict["10_2"]["vmin"], vlims_dict["10_2"]["vmax"]

# plt.colorbar(
#     cm.ScalarMappable(norm=PowerNorm(1/3., vmin=_vmin, vmax=_vmax), cmap=mycmap),
#     cax=_cax,
#     orientation="horizontal",
# )

# _cax.xaxis.set_visible(True)
# _ticks_power = range(
#     np.ceil(np.log10(_vmin)).astype(int),
#     np.floor(np.log10(_vmax)).astype(int)+1,
# )
# _cax.set_xticks([np.power(10.,_p) for _p in _ticks_power])
# _cax.set_xticklabels(['$\mathdefault{10^{%i}}$' % l for l in _ticks_power])
# # _cax.ticklabel_format(axis="x",style="sci",scilimits=(0, 0),useMathText=True)
# _cax.xaxis.set_tick_params(labelsize=6, pad=0.1, length=2)


# # # # plt.show()
# # plt.savefig("test_fig2.svg", dpi=300)